In [15]:
import numpy as np
import pandas as pd
import math
import datetime

In [16]:
def wczytanieMacierzy(rozmiar):
    file = 'Dane/data{}.txt'.format(rozmiar)
    df = pd.read_csv(file, nrows=2, header=None)
    name = df[df.index==0].values[0][0]
    n = int(df[df.index==1].values[0][0])

    df = pd.read_csv(file, skiprows=2, sep='\s+', header=None)
    arr = df.values
    arr = arr.astype('float64')
    for i in range(n):
        arr[i,i]=np.nan
    return arr

In [17]:
def ReduceMatrix(array):
    reduceValue = 0.0
    for i in range(array.shape[0]):
        minValue = np.nanmin(array[i])
        minValue = np.nan_to_num(minValue) # aby zapobiec sytuacji, gdzie minValue == nan
        reduceValue += minValue
        array[i] -= minValue
    for i in range(array.shape[1]):
        minValue = np.nanmin(array[:,i])
        minValue = np.nan_to_num(minValue)
        reduceValue += minValue
        array[:,i] -= minValue
    return reduceValue, array

In [22]:
def obliczenieCzasu(arr, n):
    start = datetime.datetime.now()
    
    first_reduce_cost, first_reduce_arr = ReduceMatrix(arr.copy())
    first_bits = 1
    vertices = np.arange(1,n)
    vertices = list(vertices)
    number_of_vertices = n-1
    my_vertice_number = 0

    index = 0

    path = list()
    path.append(0)
    all_data = list()
    all_data.append((first_reduce_cost, number_of_vertices, first_bits, index, first_reduce_arr, vertices, my_vertice_number,path))


    end_value = (1<<n) - 1 #jedynki na wszyskich miejscach, czyli każdy wierzchołek odwiedzony

    while True:
        data = min(all_data)
        old_bits = data[2]
        if old_bits == end_value:
            break

        old_reduce_cost = data[0]
        old_number_of_vertices = data[1]
        old_reduce_arr = data[4]
        old_vertices = data[5]
        parent = data[6]
        old_path = data[7]

        number_of_vertices = old_number_of_vertices - 1

        for vertice in old_vertices:
            # Przygotowanie macierzy
            new_arr = old_reduce_arr.copy()
            new_arr[parent] = np.nan
            new_arr[:,vertice] = np.nan
            new_arr[vertice,parent] = np.nan
            reduce_cost, reduce_arr = ReduceMatrix(new_arr.copy())
            reduce_cost = old_reduce_arr[parent, vertice] + old_reduce_cost + reduce_cost

            bits = old_bits
            bits |= 1 << vertice

            vertices = old_vertices.copy()
            vertices.remove(vertice) 

            path = old_path.copy()
            path.append(vertice)

            index += 1

            all_data.append((reduce_cost, number_of_vertices, bits, index, reduce_arr, vertices, vertice, path))  

        all_data.remove(data)

    data = min(all_data)
    path = data[-1].copy()
    path.append(0)
    
    duration = datetime.datetime.now() - start
    return duration.total_seconds()

In [19]:
def obliczenia(rozmiary, liczba_powtorzen):
    srednie_czasy = []
    najlepsze_czasy = []
    najgorsze_czasy = []
    
    for nr, rozmiar in enumerate(rozmiary):
        ilosc_powtorzen = liczba_powtorzen[nr]
        najlepszy_czas = 999999999
        najgorszy_czas = 0
        suma_czasow = 0
        
        for powtorzenie in range(ilosc_powtorzen):
            arr = wczytanieMacierzy(rozmiar)
            czas = obliczenieCzasu(arr, rozmiar)
            if czas < najlepszy_czas:
                najlepszy_czas = czas
            
            if czas > najgorszy_czas:
                najgorszy_czas = czas
                
            suma_czasow += czas
        
        sredni_czas = suma_czasow / ilosc_powtorzen
        najlepsze_czasy.append(najlepszy_czas)
        najgorsze_czasy.append(najgorszy_czas)
        srednie_czasy.append(sredni_czas)
    
    return najlepsze_czasy, najgorsze_czasy, srednie_czasy

In [24]:
rozmiary = [10, 11, 12, 13, 14, 15, 16, 18, 21]
liczba_powtorzen = [100,100,100,100,100,100,50,50,5]
najlepsze_czasy, najgorsze_czasy, srednie_czasy = obliczenia(rozmiary, liczba_powtorzen)

C:\Users\kulaw\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: All-NaN slice encountered
  after removing the cwd from sys.path.
C:\Users\kulaw\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: All-NaN slice encountered
  if __name__ == '__main__':


In [25]:
for i in range(len(rozmiary)):
    print('Rozmiar: ', rozmiary[i], ', liczba powtorzen: ', liczba_powtorzen[i], ', najgorsze_czasy: ', najgorsze_czasy[i], ', najlepszy czas: ', najlepsze_czasy[i], ', sredni czas: ', srednie_czasy[i])

Rozmiar:  10 , liczba powtorzen:  100 , najgorsze_czasy:  0.199495 , najlepszy czas:  0.17154 , sredni czas:  0.18169729999999998
Rozmiar:  11 , liczba powtorzen:  100 , najgorsze_czasy:  0.236366 , najlepszy czas:  0.19049 , sredni czas:  0.20037396999999996
Rozmiar:  12 , liczba powtorzen:  100 , najgorsze_czasy:  0.092787 , najlepszy czas:  0.084773 , sredni czas:  0.08902312000000004
Rozmiar:  13 , liczba powtorzen:  100 , najgorsze_czasy:  0.158577 , najlepszy czas:  0.136634 , sredni czas:  0.14454540000000007
Rozmiar:  14 , liczba powtorzen:  100 , najgorsze_czasy:  0.179545 , najlepszy czas:  0.150597 , sredni czas:  0.15894510999999997
Rozmiar:  15 , liczba powtorzen:  100 , najgorsze_czasy:  1.70843 , najlepszy czas:  1.63665 , sredni czas:  1.67092215
Rozmiar:  16 , liczba powtorzen:  50 , najgorsze_czasy:  7.364317 , najlepszy czas:  6.858659 , sredni czas:  6.9453327599999986
Rozmiar:  18 , liczba powtorzen:  50 , najgorsze_czasy:  6.581401 , najlepszy czas:  6.304169 , sr

In [26]:
PRD = []
for i in range(len(rozmiary)):
    PRD.append(100*(srednie_czasy[i] - najlepsze_czasy[i])/srednie_czasy[i])

In [27]:
df_branch_and_bound = pd.DataFrame({
    'rozmiar' : rozmiary,
    'liczba powtorzen' : liczba_powtorzen,
    'najlepszy czas' : najlepsze_czasy,
    'najgorszy czas' : najgorsze_czasy,
    'sredni czas' : srednie_czasy,
    'PRD' : PRD
})

In [28]:
df_branch_and_bound

,rozmiar,liczba powtorzen,najlepszy czas,najgorszy czas,sredni czas,PRD
0,10,100,0.171540,0.199495,0.181697,5.590232
1,11,100,0.190490,0.236366,0.200374,4.932761
2,12,100,0.084773,0.092787,0.089023,4.774176
3,13,100,0.136634,0.158577,0.144545,5.473298
4,14,100,0.150597,0.179545,0.158945,5.252197
5,15,100,1.636650,1.708430,1.670922,2.051092
6,16,50,6.858659,7.364317,6.945333,1.247943
7,18,50,6.304169,6.581401,6.377175,1.144796
8,21,5,599.286108,608.380921,603.042865,0.622967


In [29]:
df_branch_and_bound.to_csv('branch_and_bound.csv', encoding='utf-8', index=False)

17 liczył całą noc i nie doliczył, braku pamięci nie wyrzucił